In [1]:
!pip3 install beautifulsoup4
!pip3 install langchain_community
!pip3 install faiss-cpu
!pip3 install langchain
!pip3 install langchain_openai

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3

In [6]:
# from dotenv import load_dotenv
import os
#load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#print(OPENAI_API_KEY)

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)
# resp = llm.invoke("how can langsmith help with testing?")
# print(resp)

content='Langsmith can help with testing in the following ways:\n\n1. Test Automation: Langsmith can generate test scripts and code snippets to automate the testing process. It can create test cases, stubs, and mocks for different scenarios, reducing the manual effort required for testing.\n\n2. Test Data Generation: Langsmith can generate synthetic test data for different test scenarios, enabling testers to cover a wide range of inputs and edge cases. This helps in identifying bugs and vulnerabilities that might not be evident with limited real-world data.\n\n3. Test Oracles: Langsmith can generate expected outputs or oracles for a given set of inputs. This can be useful in automatically comparing the actual output of a system with the expected output, allowing for quick identification of discrepancies.\n\n4. Test Coverage Analysis: Langsmith can analyze code and provide insights into the test coverage achieved by existing test cases. It can identify areas of the code that are not ade

In [7]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])
prompt.format_prompt(input="hello")
prompt.format_messages(input="hello")

[SystemMessage(content='You are world class technical documentation writer.'),
 HumanMessage(content='hello')]

In [13]:
#define a chain by combining prompt and llm
chain = prompt | llm

#involke the chain
resp = chain.invoke({"input": "how can langsmith help with testing?"})
# This returns an AIMessage. We may need to parse it. we need its content property.
# langchain also has inbuilt parsers
resp.content

AIMessage(content='Langsmith can be a valuable tool for testing in the following ways:\n\n1. Test Automation: Langsmith enables the creation and execution of automated tests for software applications. It provides a user-friendly interface for writing test scripts using a simple yet powerful language. Testers can define test cases, input data, expected results, and assertions, making it easier to automate the testing process.\n\n2. Test Data Generation: Langsmith has built-in capabilities for generating test data. Testers can use its data generation functions to create realistic and diverse datasets to use in their tests. This ensures comprehensive testing coverage and helps identify potential issues or edge cases that might not be apparent with limited or static test data.\n\n3. Test Orchestration: Langsmith allows testers to orchestrate and manage the execution of tests across different systems and environments. It provides features to define test suites, execute tests in parallel or 

In [14]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
# combine this output parser to pur chain
chain = prompt | llm | output_parser

# invoke the chain
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can be a valuable tool in the testing process by assisting with automated testing, test case generation, and test result analysis. Here are some ways Langsmith can help with testing:\n\n1. Automated Testing: Langsmith can generate test scripts or code snippets in various programming languages to automate the testing process. It can provide specific code templates for different types of tests such as unit tests, integration tests, and end-to-end tests. These scripts can be easily integrated into testing frameworks like Selenium, JUnit, or PyTest to execute tests automatically.\n\n2. Test Case Generation: Langsmith can assist in generating test cases based on defined criteria or specifications. It can help identify edge cases, boundary values, and combinations of inputs that need to be tested. By leveraging natural language processing capabilities, Langsmith can extract requirements from documentation or user stories and generate test cases accordingly.\n\n3. Test Result Analy

# Retrival chain
Lets read from langsmith document and provide the relevant text from the
document to openAI for answering.

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
print(len(docs))
# print(docs)

embeddings = OpenAIEmbeddings()

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
print("number of docuemnts=",len(documents))
vector = FAISS.from_documents(documents, embeddings)


1
number of docuemnts= 5


In [8]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

# Running this document chain manually by passing some context text

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})


'Langsmith can help with testing by allowing users to visualize test results.'

In [9]:
# Create retrival chain by adding our vector store as a retiever. 
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# Conversation Retrieval Chain

we will create a new chain. This chain will take in the most recent input (input) and the conversation history (chat_history) and use an LLM to generate a search query

In [11]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

# test it bypassing a chat_history
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
 Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookRelease NotesOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past

In [12]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', 'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en'}),
  Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookRelease NotesOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is

# Agent

In [12]:
! pip3 install -U langchain-community tavily-python
! pip3 install langchainhub 

9899.72s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


9906.86s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [13]:
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

search = TavilySearchResults()

tools = [retriever_tool, search]

from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "how can langsmith help with testing?"})

agent_executor.invoke({"input": "what is the weather in Hyderabad?"})



> Entering new AgentExecutor chain...
LangSmith can help with testing in several ways:

1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.

2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.

3. Test Coverage Analysis: LangSmith can analyze the code coverage of the test cases to determine which parts of the software have been exercised during testing. This helps in identifying areas of the code that have not been adequately tested and may require additional test cases.

4. Test Prioritization: LangSmith can prioritize the exec

{'input': 'what is the weather in Hyderabad?',
 'output': 'I found some information about the weather in Hyderabad. Here are a few sources you can check for more details:\n\n1. [WhereAndWhen.net](https://www.whereandwhen.net/when/central-and-south-asia/india/hyderabad/february/): This website provides seasonal average climate and temperature information for Hyderabad in February.\n\n2. [WeatherSpark](https://weatherspark.com/h/y/109450/2024/Historical-Weather-during-2024-in-Hyderabad-India): This website offers historical weather data for Hyderabad in 2024, including temperature, cloud cover, and wind information.\n\n3. [Climate-Data.org](https://en.climate-data.org/asia/india/hyderabad/hyderabad-2801/t/february-2/): This website provides weather averages for Hyderabad in February, including average temperature.\n\n4. [Weather25.com](https://www.weather25.com/asia/india/andhra-pradesh/hyderabad): This website offers 14-day weather forecasts for Hyderabad and provides information about 